# EIA Project

Following a summary of the project. For more details, go to the report

**About the EIA**

The U.S. Energy Information Administration (EIA) is a government agency within the U.S. Department of Energy that collects, analyzes, and publishes independent and impartial energy information.
Its mission is to promote sound policymaking, efficient markets, and public understanding of energy and its interaction with the economy and the environment.

The EIA provides comprehensive data on petroleum, natural gas, coal, electricity, renewables, and nuclear energy with production, consumption, imports/exports, inventories, prices, regular reports such as the Weekly Petroleum Status Report (WPSR), Short-Term Energy Outlook, and Annual Energy Outlook.

For oil markets, the Weekly Petroleum Status Report is a key source of market fundamentals, summarizing U.S. crude oil and petroleum product supply, demand, and inventory levels — data closely followed by traders, analysts, and policy makers for its potential to influence market sentiment and decision-making.

Source: https://www.eia.gov/

**Business Problem**

An energy trading desk wants to understand which petroleum market fundamentals have the most influence on WTI spot prices, so they can prioritize monitoring and risk assessments on the most impactful indicators.

By identifying and quantifying these relationships, the desk can improve market awareness, refine hedging strategies, and enhance the efficiency of internal reporting.

**Business Question**

Which petroleum market fundamentals reported in the EIA’s Weekly Petroleum Status Report have the strongest historical influence on weekly average WTI spot prices?

**Stakeholders**

-	Trading desk: to allocate research resources toward the most relevant fundamentals.
- Market analysts: to focus daily commentary on key drivers.
- Risk management team: to incorporate key fundamentals into stress tests and VaR scenarios.
- Executives: to understand high-level market dynamics without deep technical knowledge.


**Business Objectives**

1.	Build a machine learning model to explain weekly average WTI spot price using petroleum market fundamentals from the EIA’s Weekly Petroleum Status Report (WPSR).
2.	Rank the importance of each feature in explaining historical price variation.
3.	Deliver actionable recommendations on which fundamentals to track most closely for market monitoring.


**Metrics**

Technical:
-	R² and MAE for in-sample model fit (to confirm model quality).
-	Feature importance rankings and interpretability metrics.

Business:

-	Clarity of insights for non-technical stakeholders.
-	Ability to produce a prioritized driver list that matches market intuition.


**Type of Modeling**

•	Regression (supervised learning) with emphasis on explanatory power.

•	Potential algorithms: Linear Regression, Lasso/Ridge, Random Forest Regressor, XGBoost Regressor.

•	Focus on interpretability alongside model fit quality (R², MAE).

**Dataset**

Source: U.S. Energy Information Administration (EIA) — Weekly Petroleum Status Report (WPSR) and related datasets.

Frequency: Weekly.

Period: ~30 years (if available) for robust statistical learning.

Target variable: EIA Weekly Average WTI spot price (Cushing, OK).

Features:

-	U.S. crude oil stocks (ex-SPR)
-	Total motor gasoline stocks
-	Distillate fuel oil stocks
-	Total product supplied
-	Refinery capacity utilization (%)
-	Crude oil production (U.S.)
-	Net imports/exports of crude and products
-	Seasonal indicator (week of year)

**Evaluation Plan**

1.	EDA to explore relationships between fundamentals and price.
2.	Model fitting with cross-validation for robustness.
3.	Feature importance analysis.
4.	Business interpretation — map statistical importance to actionable insights.

# Imports + Config

In [ ]:
import os, sys, json, textwrap, getpass, random
from dateutil import parser as dtparser
import time
import requests
import pandas as pd
from typing import Dict, Tuple

We use a hidden prompt so the key is kept out of the notebook and Git history. This aligns with good security practices and the EIA TOS.

In [ ]:
# Upload the API key
os.environ["EIA_API_KEY"] = getpass.getpass("Paste your EIA API key (hidden): ")

Paste your EIA API key (hidden): ··········


We save raw files to a consistent folder structure and predefine a rolling 20-year cutoff so later modeling can focus on a modern regime while keeping reproducibility.

In [ ]:
# Verify the key is loaded
API_KEY = os.environ.get("EIA_API_KEY")
assert API_KEY and len(API_KEY) > 20, "API key not set. Re-run Step 2 and paste your key."
print("API key loaded")

API key loaded


In [ ]:
# Set output folders
BASE_DIR = "/content"               # Colab working dir
RAW_DIR  = f"{BASE_DIR}/data/raw"   # where we'll save raw CSVs
os.makedirs(RAW_DIR, exist_ok=True)

# Rolling 20-year cutoff (dynamic)
CUTOFF_20Y = pd.Timestamp.today().normalize() - pd.DateOffset(years=20)
print("Saving raw files to:", RAW_DIR)
print("20-year cutoff:", CUTOFF_20Y.date())

Saving raw files to: /content/data/raw
20-year cutoff: 2005-08-12


A single registry (dictionary) maps human-readable names to EIA series IDs. Adding new data later is as simple as adding one line here.

In [ ]:
# v2 endpoint that serves legacy series IDs (we're not using legacy API)
EIA_SERIES_URL = "https://api.eia.gov/v2/seriesid/{series_id}?api_key={api_key}"

# Friendly names → EIA series IDs
SERIES = {
    # Target (weekly average WTI, $/bbl)
    "WTI_weekly_avg_usd_per_bbl": "PET.RWTC.W",

    # Core WPSR fundamentals (weekly)
    "crude_stocks_excl_SPR_kbbl": "WCESTUS1",
    "gasoline_stocks_kbbl":       "WGTSTUS1",
    "distillate_stocks_kbbl":     "WDISTUS1",
    "refinery_utilization_pct":   "WPULEUS3",
    "crude_field_prod_kbd":       "WCRFPUS2",
    "net_imports_crude_prod_kbd": "WTTNTUS2",
    "total_products_supplied_kbd":"WRPUPUS2",
}

print(f"{len(SERIES)} series configured:\n- " + "\n- ".join(SERIES.keys()))

8 series configured:
- WTI_weekly_avg_usd_per_bbl
- crude_stocks_excl_SPR_kbbl
- gasoline_stocks_kbbl
- distillate_stocks_kbbl
- refinery_utilization_pct
- crude_field_prod_kbd
- net_imports_crude_prod_kbd
- total_products_supplied_kbd


**Helper function to fetch a single series**

The EIA API provides many different series IDs (e.g., WTI prices, crude stocks, gasoline stocks). Rather than writing repetitive request code for each series, the fetch_series() function encapsulates the logic in a reusable way. This allows us to call it repeatedly with different series IDs and automatically receive clean, pre-formatted pandas DataFrames.

This design is modular, maintainable, and easy to debug: if the API endpoint or data format changes, updates need to be made in only one place. The function also incorporates retry logic and error handling to ensure reliability when interacting with the EIA API.

In [ ]:
def fetch_series(series_id: str, api_key: str, retries: int = 8, pause: float = 1.0) -> pd.DataFrame:
    """
    Fetch one EIA series via v2 /seriesid (no legacy fallback).
    Retries on transient errors (429/5xx) with exponential backoff.
    Returns ['period','value','series_id'] sorted ascending.
    """
    last_ex = None
    for attempt in range(1, retries + 1):
        try:
            url = EIA_SERIES_URL.format(series_id=series_id, api_key=api_key)
            r = requests.get(url, timeout=30)
            status = r.status_code
            if status in (429, 500, 502, 503, 504):
                raise requests.HTTPError(f"HTTP {status}")
            r.raise_for_status()

            js = r.json()
            rows = js["response"]["data"]
            if not rows:
                raise ValueError(f"No data returned for {series_id}")

            df = pd.DataFrame(rows)[["period","value"]].copy()
            df["period"] = pd.to_datetime(df["period"])
            df["series_id"] = series_id
            return df.sort_values("period").reset_index(drop=True)

        except Exception as ex:
            last_ex = ex
            sleep_s = (pause * (2 ** (attempt - 1))) + random.uniform(0, 0.5)
            print(f"[{series_id}] attempt {attempt}/{retries} failed: {ex} → retrying in {sleep_s:.1f}s")
            time.sleep(min(sleep_s, 60.0))

    raise RuntimeError(f"Failed to fetch {series_id} after {retries} attempts; last: {last_ex}")

**Batch pull helper: tidy + wide outputs**

Report note: We pull all series, save one CSV per series (traceability), then pivot into a single wide table (model-ready) with one row per week and one column per variable.

In [ ]:
def pull_series_map(series_map: Dict[str, str], api_key: str, save_dir: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Pulls all series, saving per-series CSVs; uses cache if file exists; skips failures.
    Returns (tidy_df, wide_df).
    """
    all_tidy = []
    for nice_name, sid in series_map.items():
        out_path = os.path.join(save_dir, f"{nice_name}.csv")
        try:
            if os.path.exists(out_path):
                df = pd.read_csv(out_path, parse_dates=["period"])
                print(f"Loaded cached {nice_name} ← {out_path} ({len(df)} rows)")
            else:
                df = fetch_series(sid, api_key)
                df.to_csv(out_path, index=False)
                print(f"Saved {nice_name} → {out_path} ({len(df)} rows)")
            df["nice_name"] = nice_name
            all_tidy.append(df)
        except Exception as ex:
            print(f"SKIP: {nice_name} due to error: {ex}")
            # move on; you can re-run later to fill missing series
            continue

    tidy = pd.concat(all_tidy, ignore_index=True)
    wide  = tidy.pivot_table(index="period", columns="nice_name", values="value", aggfunc="first").sort_index()
    return tidy, wide

**Run the pull, save full + 20-year combined files**

Report note: We keep a full combined file and also a 20-year subset for the modeling scope. This supports both reproducibility and sensitivity analysis later.

In [ ]:
tidy, wide = pull_series_map(SERIES, API_KEY, RAW_DIR)
print("Combined shape (full):", wide.shape)
display(wide.tail(3))

Loaded cached WTI_weekly_avg_usd_per_bbl ← /content/data/raw/WTI_weekly_avg_usd_per_bbl.csv (2066 rows)
[WCESTUS1] attempt 1/8 failed: HTTP 500 → retrying in 1.5s
[WCESTUS1] attempt 2/8 failed: HTTP 500 → retrying in 2.0s
[WCESTUS1] attempt 3/8 failed: HTTP 500 → retrying in 4.4s
[WCESTUS1] attempt 4/8 failed: HTTP 500 → retrying in 8.4s
[WCESTUS1] attempt 5/8 failed: HTTP 500 → retrying in 16.3s
[WCESTUS1] attempt 6/8 failed: HTTP 500 → retrying in 32.2s
[WCESTUS1] attempt 7/8 failed: HTTP 500 → retrying in 64.3s
[WCESTUS1] attempt 8/8 failed: HTTP 500 → retrying in 128.0s
SKIP: crude_stocks_excl_SPR_kbbl due to error: Failed to fetch WCESTUS1 after 8 attempts; last: HTTP 500
[WGTSTUS1] attempt 1/8 failed: HTTP 500 → retrying in 1.4s
[WGTSTUS1] attempt 2/8 failed: HTTP 500 → retrying in 2.4s
[WGTSTUS1] attempt 3/8 failed: HTTP 500 → retrying in 4.3s
[WGTSTUS1] attempt 4/8 failed: HTTP 500 → retrying in 8.1s
[WGTSTUS1] attempt 5/8 failed: HTTP 500 → retrying in 16.4s
[WGTSTUS1] attempt

nice_name,WTI_weekly_avg_usd_per_bbl
period,
2025-07-18,68.07
2025-07-25,67.11
2025-08-01,69.58
